In [ ]:
##Import Dependencies

In [1]:
password = 'slytherin10946'
username = 'vigrose'

In [2]:
import pandas as pd
import pyodbc
import pgeocode
import numpy as np

In [ ]:
##Connect to the database

In [3]:
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [ ]:
##Read in all latest insurance addresses

In [4]:
sql_query = \
    """ 
    SELECT DISTINCT
    A.PARTY_ID, 
    K.KEY_VAL AS ME, 
    A.POST_CD_ID,
    A.FROM_DT,
    C.PURPOSE_TYPE_DESC AS ADDR_TYPE, 
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP
    FROM 
    AMAEDW.PARTY_KEY K, AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.POST_CD P, AMAEDW.STATE S 
    WHERE  
    A.THRU_DT IS NULL
    AND
    K.PARTY_ID = A.PARTY_ID 
    AND 
    K.KEY_TYPE_ID = 18 
    AND 
    K.ACTIVE_IND = 'Y'
    AND
    C.PURPOSE_TYPE_ID IN (1)
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    ORDER BY A.FROM_DT DESC;
    """

In [5]:
insurance = pd.read_sql(con=AMAEDW, sql=sql_query)
insurance = insurance.drop_duplicates(subset = 'PARTY_ID', keep='first'
print(insurance.shape))
insurance.head()

(1599446, 11)


,PARTY_ID,ME,POST_CD_ID,FROM_DT,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP
0,1986694,02312981460,19071820,2020-03-17 01:33:34,Billing,794 Industry Rd,,,New Sharon,ME,04955
1,6448098,05605160532,21482437,2020-03-17 01:33:34,Billing,3181 SW Sam Jackson Park Rd,Mail Code L475,,Portland,OR,97239
3,2892331,58302790173,23172284,2020-03-17 01:33:34,Billing,11050 Mount Belvedere Blvd,USA Meddac,Attn: Credentials Office,Fort Drum,NY,13602
4,2152521,02312871133,25501403,2020-03-17 01:33:34,Billing,10475 Centurion Pkwy N Ste 220,Southeast Orthopedic Specialists,,Jacksonville,FL,32256
5,2051466,02312861111,26441934,2020-03-17 01:33:34,Billing,701 N Clayton St,Mob Suite 505,,Wilmington,DE,19805


In [8]:
insurance_2 = insurance.drop_duplicates(subset = 'PARTY_ID', keep='first')

In [9]:
insurance_2

,PARTY_ID,ME,POST_CD_ID,FROM_DT,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP
0,2061002,02106861083,11894260,2020-03-12 00:09:45.000000,Billing,5312 S Ingleside Ave,,,Chicago,IL,60615
1,5838100,03901140692,15585246,2020-03-12 00:09:45.000000,Billing,2747 NE Conners Ave,Bend Dermatology Clinic,,Bend,OR,97701
2,2175577,02834970565,16388430,2020-03-12 00:09:45.000000,Billing,2450 Riverside Ave,,,Minneapolis,MN,55454
3,2456010,05605911325,19559500,2020-03-12 00:09:45.000000,Billing,610 NW 11th St,Good Shepherd Medical Center,,Hermiston,OR,97838
4,3563145,04815081221,20917282,2020-03-12 00:09:45.000000,Billing,10300 NE Hancock St,Vibra Specialty Hospital,,Portland,OR,97220
...,...,...,...,...,...,...,...,...,...,...,...
14212204,2016155,01606689957,11731201,2005-12-16 20:39:34.391274,Billing,PO Box 8128,,,Chicago,IL,60680
14223074,1941127,01606769926,11799144,2005-12-16 20:38:26.787136,Billing,PO Box 8208,,,Chicago,IL,60680
14233372,2057340,02401980221,13364686,2005-12-16 20:37:10.318382,Billing,107 Avenue Louis Pasteur,Vanderbilt Hall Bx 249,,Boston,MA,02115
14304013,1810489,00512010695,9425824,2005-12-16 20:21:59.120256,Billing,1021 W Cypress Ave,,,Redlands,CA,92373


In [ ]:
#Read all latest PPMA & POLO addresses

In [16]:
sql_query_PPMA = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PP'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
        '''

In [17]:
PPMA = pd.read_sql(con=AMAEDW, sql=sql_query_PPMA)
print(PPMA.shape)
PPMA.head()

(1602649, 9)


,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT
0,1885202,30695370,969 SE Central Pkwy,McKay Dermatology & MedSpa,,Stuart,FL,34994,2020-03-11 21:11:05.040363
1,4581087,30695479,4 Ridge Cliff Ct,Irmo,,Irmo,SC,29063,2020-03-11 21:11:04.840764
2,9612892,30695495,817 Castlemaine Ct,,,Birmingham,AL,35226,2020-03-11 21:11:04.835095
3,5170034,30695452,"Calle Antulios, F-10, Urb. Estancias de",00727,,Caguas,PR,00727,2020-03-11 21:11:04.829956
4,13477040,30695368,2550 SW 18th Terrace Apt 2002,,,Fort Lauderdale,FL,33315,2020-03-11 21:11:04.825449


In [14]:
sql_query_POLO = \
    '''
    SELECT DISTINCT 
    A.PARTY_ID, 
    A.POST_CD_ID,
    P.ADDR_1, 
    P.ADDR_2, 
    P.ADDR_3, 
    P.CITY, 
    S.SRC_STATE_CD AS STATE_CD, 
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY
    FROM 
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE 
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID 
    AND
    P.STATE_ID = S.STATE_ID
    AND 
    A.THRU_DT IS NULL
    ORDER BY A.FROM_DT DESC
    '''

In [15]:
POLO = pd.read_sql(con=AMAEDW, sql=sql_query_POLO)
print(POLO.shape)
POLO.head()

(1163784, 10)


,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT,POST_KEY
0,3550162,30695490,1529 W State Route 366,Federal Correctional Institution Safford,,Safford,AZ,85546,2020-03-11 21:11:04.804553,24418872
1,4506247,30695478,42 N Spring Dr,,,Jackson,TN,38305,2020-03-11 21:11:04.774658,24418789
2,2738500,30695346,901 N Galloway Ave,Ste 109,,Mesquite,TX,75149,2020-03-11 21:11:04.746572,24418714
3,7624178,30695482,9800 Shelbyville Rd,Ste 220,Attn: Sharon Hilton,Louisville,KY,40223,2020-03-11 21:11:04.740698,24418712
4,6167780,30695504,1101 Madison St,Ste 950,Seattle OB/GYN,Seattle,WA,98104,2020-03-11 21:11:04.724577,24418689


In [ ]:
POLO = POLO.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(POLO.shape)
PPMA= PPMA.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(PPMA.shape)

In [ ]:
##Read latest PPD

In [5]:
path = "..\PhysicianProfessionalDataFile_20191012_062051504"
cols = [
        'ME',
        'RECORD_ID',
        'UPDATE_TYPE',
        'ADDRESS_TYPE',
        'MAILING_NAME',
        'LAST_NAME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'SUFFIX',
        'MAILING_LINE_1',
        'MAILING_LINE_2',
        'CITY',
        'STATE',
        'ZIP',
        'SECTOR',
        'CARRIER_ROUTE',
        'ADDRESS_UNDELIVERABLE_FLAG',
        'FIPS_COUNTY',
        'FIPS_STATE',
        'PRINTER_CONTROL_CODE',
        'PC_ZIP',
        'PC_SECTOR',
        'DELIVERY_POINT_CODE',
        'CHECK_DIGIT',
        'PRINTER_CONTROL_CODE_2',
        'REGION',
        'DIVISION',
        'GROUP',
        'TRACT',
        'SUFFIX_CENSUS',
        'BLOCK_GROUP',
        'MSA_POPULATION_SIZE',
        'MICRO_METRO_IND',
        'CBSA',
        'CBSA_DIV_IND',
        'MD_DO_CODE',
        'BIRTH_YEAR',
        'BIRTH_CITY',
        'BIRTH_STATE',
        'BIRTH_COUNTRY',
        'GENDER',
        'TELEPHONE_NUMBER',
        'PRESUMED_DEAD_FLAG',
        'FAX_NUMBER',
        'TOP_CD',
        'PE_CD',
        'PRIM_SPEC_CD',
        'SEC_SPEC_CD',
        'MPA_CD',
        'PRA_RECIPIENT',
        'PRA_EXP_DT',
        'GME_CONF_FLG',
        'FROM_DT',
        'TO_DT',
        'YEAR_IN_PROGRAM',
        'POST_GRADUATE_YEAR',
        'GME_SPEC_1',
        'GME_SPEC_2',
        'TRAINING_TYPE',
        'GME_INST_STATE',
        'GME_INST_ID',
        'MEDSCHOOL_STATE',
        'MEDSCHOOL_ID',
        'MEDSCHOOL_GRAD_YEAR',
        'NO_CONTACT_IND',
        'NO_WEB_FLAG',
        'PDRP_FLAG',
        'PDRP_START_DT',
        'POLO_MAILING_LINE_1',
        'POLO_MAILING_LINE_2',
        'POLO_CITY',
        'POLO_STATE',
        'POLO_ZIP',
        'POLO_SECTOR',
        'POLO_CARRIER_ROUTE',
        'MOST_RECENT_FORMER_LAST_NAME',
        'MOST_RECENT_FORMER_MIDDLE_NAME',
        'MOST_RECENT_FORMER_FIRST_NAME',
        'NEXT_MOST_RECENT_FORMER_LAST',
        'NEXT_MOST_RECENT_FORMER_MIDDLE',
        'NEXT_MOST_RECENT_FORMER_FIRST'
    ]
ppd = pd.read_csv(path, names=cols, sep='|', encoding='IBM437', index_col=False, dtype=object)
PPD = ppd[['ME','MAILING_NAME', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME']]

In [ ]:
##Merge PPMA, POLO, PPD, and Insurance by ME and PARTY_ID

In [ ]:
ppma_polo = pd.merge(PPMA, POLO, on = 'PARTY_ID', how= 'outer', suffixes = ('_PPMA', '_POLO'))

In [11]:
ppd_insurance_2 = pd.merge(insurance_2, PPD, on = 'ME')

In [19]:
INSURANCE_2 = ppd_insurance_2.drop(columns=['PARTY_ID','ME', 'POST_CD_ID'])

In [20]:
INSURANCE_2.to_csv('insurance_current.csv')

In [9]:
INSURANCE.to_csv('insurance_historical.gz', compression='gzip')

In [ ]:
PPD.shape

In [ ]:
ppd_insurance.shape

In [ ]:
insurance_polo = pd.merge(ppd_insurance, POLO, on = 'PARTY_ID', suffixes = ('_insurance', '_POLO'))

In [ ]:
insurance_ppma = pd.merge(ppd_insurance, PPMA, on = 'PARTY_ID', suffixes = ('_insurance', '_PPMA'))

In [ ]:
insurance_ppma_polo = pd.merge(ppd_insurance, ppma_polo, on = 'PARTY_ID')

In [ ]:
insurance_ppma_polo.shape

In [ ]:
insurance_ppma_polo = insurance_ppma_polo.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(insurance_ppma_polo.shape)

In [ ]:
##Summarize the address differences and generate a table of mismat

In [ ]:
#Define summarizing function (PPMA)
def sum_it_up_ppma(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT_insurance > row.FROM_DT_PPMA:
            if row.POST_CD_ID_PPMA != row.POST_CD_ID_insurance:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
                new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
                new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
                new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                new_dict['ZIP_insurance'] = row.ZIP_insurance
                new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                new_dict['CITY_insurance'] = row.CITY_insurance
                new_dict['CITY_PPMA'] = row.CITY_PPMA
                new_dict['DATE_insurance'] = row.FROM_DT_insurance
                new_dict['DATE_PPMA'] = row.FROM_DT_PPMA
                new_dict['STATE_insurance'] = row.STATE_CD_insurance
                new_dict['STATE_PPMA'] = row.STATE_CD_PPMA
                dict_list.append(new_dict)

    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs do not match 
    ''')
    return(dict_list)

In [ ]:
#Define summarizing function (POLO)
def sum_it_up_polo(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT_insurance > row.FROM_DT_POLO:
            if row.POST_CD_ID_POLO == row.POST_CD_ID_insurance:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
                new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
                new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
                new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                new_dict['ZIP_insurance'] = row.ZIP_insurance
                new_dict['ZIP_POLO'] = row.ZIP_POLO
                new_dict['CITY_insurance'] = row.CITY_insurance
                new_dict['CITY_POLO'] = row.CITY_POLO
                new_dict['DATE_insurance'] = row.FROM_DT_insurance
                new_dict['DATE_POLO'] = row.FROM_DT_POLO
                new_dict['STATE_insurance'] = row.STATE_CD_insurance
                new_dict['STATE_POLO'] = row.STATE_CD_POLO
                dict_list.append(new_dict)

    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs match 
    ''')
    return(dict_list)

In [ ]:
#Define summarizing function (ALL)
def sum_it_up(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.FROM_DT > row.FROM_DT_PPMA and row.FROM_DT > row.FROM_DT_POLO:
            if row.POST_CD_ID_POLO != row.POST_CD_ID and row.POST_CD_ID != row.POST_CD_ID_PPMA:
                cd_count +=1
            
                new_dict = {}
                new_dict['ME'] = row.ME
                new_dict['ADDR_1_insurance'] = row.ADDR_1
                new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                new_dict['ADDR_2_insurance'] = row.ADDR_2
                new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                new_dict['ADDR_3_insurance'] = row.ADDR_3
                new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                new_dict['ZIP_insurance'] = row.ZIP
                new_dict['ZIP_POLO'] = row.ZIP_POLO
                new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                new_dict['CITY_insurance'] = row.CITY
                new_dict['CITY_POLO'] = row.CITY_POLO
                new_dict['CITY_PPMA'] = row.CITY_PPMA
                new_dict['DATE_insurance'] = row.FROM_DT
                new_dict['DATE_POLO'] = row.FROM_DT_POLO
                new_dict['DATE_PPMA'] = row.FROM_DT_PPMA
                new_dict['STATE_insurance'] = row.STATE_CD
                new_dict['STATE_POLO'] = row.STATE_CD_POLO
                new_dict['STATE_PPMA'] = row.STATE_CD_PPMA
                dict_list.append(new_dict)

                if row.ADDR_1 != row.ADDR_1_PPMA and row.ADDR_1 != row.ADDR_1_POLO:
                    add_count +=1
                if row.CITY != row.CITY_PPMA and row.CITY != row.CITY_POLO:
                    city_count +=1
                if row.ZIP != row.ZIP_PPMA and row.ZIP != row.ZIP_POLO:
                    zip_count +=1
                if row.STATE_CD != row.STATE_CD_PPMA and row.STATE_CD != row.STATE_CD_POLO:
                    state_count +=1
    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs do not match 
    {add_count/cd_count*100}% ({add_count}) addresses do not match
    {zip_count/cd_count * 100}% ({zip_count})zipcodes do not match
    {state_count/cd_count * 100}% ({state_count})states do not match
    {city_count/cd_count * 100}% ({city_count})cities do not match
    ''')
    return(dict_list)

In [ ]:
insurance_mismatch = sum_it_up(insurance_ppma_polo)
insurance_mismatch_df = pd.DataFrame(insurance_mismatch)

In [ ]:
polo_match = sum_it_up_polo(insurance_polo)
polo_match_df = pd.DataFrame(polo_match)

In [ ]:
ppma_mismatch = sum_it_up_ppma(insurance_ppma)
ppma_mismatch_df = pd.DataFrame(ppma_mismatch)

In [ ]:
pd.merge(ppma_match_df, polo_match_df, on = ['ME','ADDR_1_insurance','ADDR_2_insurance','ADDR_3_insurance', 'CITY_insurance', 'DATE_insurance', 'STATE_insurance', 'ZIP_insurance'])

In [ ]:
#Define summarizing function
def narrow_it_down(df, address_type):
    dict_list = []
    for row in df.itertuples():
        if address_type == 'PPMA'
            if row.STATE_insurance != row.STATE_PPMA:
                if row.CITY_insurance != row.CITY_PPMA:
                    if row.ADDR_1_insurance != row.ADDR_1_PPMA:
                        if row.ZIP_insurance != row.ZIP_PPMA:
                            new_dict = {}
                            new_dict['ME'] = row.ME
                            new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance

                            new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                            new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance

                            new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                            new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance

                            new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                            new_dict['ZIP_insurance'] = row.ZIP_insurance

                            new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                            new_dict['CITY_insurance'] = row.CITY_insurance

                            new_dict['CITY_PPMA'] = row.CITY_PPMA
                            new_dict['DATE_insurance'] = row.DATE_insurance

                            new_dict['DATE_PPMA'] = row.DATE_PPMA
                            new_dict['STATE_insurance'] = row.STATE_insurance

                            new_dict['STATE_PPMA'] = row.STATE_PPMA
                            dict_list.append(new_dict)
                            
        if address_type == 'POLO'
            if row.STATE_insurance != row.STATE_POLO:
                if row.CITY_insurance != row.CITY_POLO:
                    if row.ADDR_1_insurance != row.ADDR_1_POLO:
                        if row.ZIP_insurance != row.ZIP_POLO:
                            new_dict = {}
                            new_dict['ME'] = row.ME
                            new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance

                            new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                            new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance

                            new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                            new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance

                            new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                            new_dict['ZIP_insurance'] = row.ZIP_insurance

                            new_dict['ZIP_POLO'] = row.ZIP_POLO
                            new_dict['CITY_insurance'] = row.CITY_insurance

                            new_dict['CITY_POLO'] = row.CITY_POLO
                            new_dict['DATE_insurance'] = row.DATE_insurance

                            new_dict['DATE_POLO'] = row.DATE_POLO
                            new_dict['STATE_insurance'] = row.STATE_insurance

                            new_dict['STATE_POLO'] = row.STATE_POLO
                            dict_list.append(new_dict)          
    NEW_DF=pd.DataFrame(dict_list)
    return(NEW_DF)

In [ ]:
#Define summarizing function
def narrow_it_down(df):
    dict_list = []
    for row in df.itertuples():
        if row.STATE_insurance != row.STATE_PPMA and row.STATE_insurance != row.STATE_POLO:
            if row.CITY_insurance != row.CITY_PPMA and row.CITY_insurance != row.CITY_POLO:
                if row.ADDR_1_insurance != row.ADDR_1_PPMA and row.ADDR_1_insurance != row.ADDR_1_POLO:
                    if row.ZIP_insurance != row.ZIP_PPMA and row.ZIP_insurance != row.ZIP_POLO:
                        new_dict = {}
                        new_dict['ME'] = row.ME
                        new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
                        new_dict['ADDR_1_POLO'] = row.ADDR_1_POLO
                        new_dict['ADDR_1_PPMA'] = row.ADDR_1_PPMA
                        new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
                        new_dict['ADDR_2_POLO'] = row.ADDR_2_POLO
                        new_dict['ADDR_2_PPMA'] = row.ADDR_2_PPMA
                        new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
                        new_dict['ADDR_3_POLO'] = row.ADDR_3_POLO
                        new_dict['ADDR_3_PPMA'] = row.ADDR_3_PPMA
                        new_dict['ZIP_insurance'] = row.ZIP_insurance
                        new_dict['ZIP_POLO'] = row.ZIP_POLO
                        new_dict['ZIP_PPMA'] = row.ZIP_PPMA
                        new_dict['CITY_insurance'] = row.CITY_insurance
                        new_dict['CITY_POLO'] = row.CITY_POLO
                        new_dict['CITY_PPMA'] = row.CITY_PPMA
                        new_dict['DATE_insurance'] = row.DATE_insurance
                        new_dict['DATE_POLO'] = row.DATE_POLO
                        new_dict['DATE_PPMA'] = row.DATE_PPMA
                        new_dict['STATE_insurance'] = row.STATE_insurance
                        new_dict['STATE_POLO'] = row.STATE_POLO
                        new_dict['STATE_PPMA'] = row.STATE_PPMA
                        dict_list.append(new_dict)
    NEW_DF=pd.DataFrame(dict_list)
    return(NEW_DF)

In [ ]:
insurance_no_matches  = narrow_it_down(insurance_mismatch_df)

In [ ]:
no_matches = narrow_it_down(mismatch_df)

In [ ]:
def get_zip_distance(df, DIST=pgeocode.GeoDistance('US')):
    distance = DIST.query_postal_code(df[],df[])*0.621371
    return(distance)

In [ ]:
#Define function to get distances between zipcodes in dataframe
def get_zip_distances(df):    
    distances = []
    TOTAL = df.shape[0]
    print(TOTAL)
    count =0
    DIST=pgeocode.GeoDistance('US')
    dict_list = []
    num_list =[]
    for row in df.itertuples():
        try:
            new_dict={}
            new_dict["ME"]=row.ME
            count += 1
            if int(count/TOTAL*100) in list(range(1,100)) and int(count/TOTAL*100) not in num_list:
                num_list.append(int(count/TOTAL*100))
                print (f'{int(count/TOTAL*100)}% done!')
            try:
                mf = int(row.ZIP_POLO)
    #             print(mf)
                ins = int(row.ZIP_insurance)
    #             print(ins)
            except:
                mf='nan'
                ins='nan'
        #     print(f'{mf} {ins}')
            try:
                ZIP_distance = get_zip_distance(mf,ins)
    #             print(ZIP_distance)
                if np.isnan(ZIP_distance)==False:
                    distances.append(ZIP_distance)     
                else:
                    print(f'{count}: {row.ME} \nMasterfile:{mf} \nInsurance:{ins}')
            except:
                ZIP_distance = 'nan'
                print(f'{count}: {row.ME} \nMasterfile:{mf} \nInsurance:{ins}')
            new_dict['ZIP_distance']=ZIP_distance
            dict_list.append(new_dict)
        # print(distances2)
        except KeyboardInterrupt:
            print("Interrupting...")
            return(dict_list, distances)
        try:
            avg = sum(distances)/len(distances)
        except:
            avg=0
    print(f'Average distance between insurance and masterfile addressses is {avg} miles.')
    return(dict_list, distances)

In [ ]:
zip_dicts_polo, zip_dists_polo = get_zip_distances(no_matches)

In [ ]:
def get_zip_distance(zip1,zip2):
    zip1 = clean_zipcode(zip1)
    zip2 = clean_zipcode(zip2)
    Distance = DIST.query_postal_code(zip1, zip2)*0.621371
    return(Distance)

In [ ]:
zippos = zip_dicts+zip_dicts2 + zip_dicts3
pd.DataFrame(zippos)

In [ ]:
no_matches.shape[0]

In [ ]:
def clean_zipcode(zipcode):
    zipcode=str(zipcode)
    zipcode = zipcode.replace(" ","")
    zipcode = zipcode.replace(".0","")
    if len(zipcode)==3:
        zipcode = "00"+zipcode
    if len(zipcode)==4: 
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return(zipcode)

In [ ]:
no_matches.to_csv('no_matches.csv')

In [ ]:
polo_dist = pd.DataFrame(zip_dicts_polo)

In [ ]:
ppma_dist=pd.DataFrame(zippos)

In [ ]:
no_matches = no_matches.drop(columns=['POLO_distance'])
with_zips = pd.merge(no_matches, polo_dist, on='ME', how='outer')

In [ ]:
with_zips = pd.merge(with_zips, ppma_dist, on='ME', how='outer', suffixes = ('_polo', '_ppma'))

In [ ]:
with_zips['ZIP_distance_polo'] = with_zips['ZIP_distance_polo'].astype(float)
with_zips['ZIP_distance_ppma'] = with_zips['ZIP_distance_ppma'].astype(float)

In [ ]:
far_places = with_zips[(with_zips['ZIP_distance_polo']>300) & (with_zips['ZIP_distance_ppma']>300)]

In [ ]:
far_places[['DATE_POLO', 'DATE_PPMA', 'DATE_insurance']]

In [ ]:
all_addresses[['PARTY_ID','ME']]

In [ ]:
small_table = all_addresses[['PARTY_ID','POST_CD_ID', 'ME']]

In [ ]:
with_ids = pd.merge(small_table, far_places, on='ME')

In [ ]:
pd.merge(with_ids, long_time, on=['PARTY_ID', 'POST_CD_ID'])

In [ ]:
with_ids.to_csv('Insurance_Updates.csv', index=False)

In [ ]:
far_places

In [ ]:
ppma_match_df.to_csv('PPMA_Insurance_Matches.csv',index=False)

In [ ]:
ppma_mismatch_df.to_csv('PPMA_Insurance_POST_CD_ID_Mismatches.csv',index=False)

In [ ]:
ppma_extreme_mismatches = narrow_it_down_ppma(ppma_mismatch_df)


In [ ]:
print(f'{ppma_extreme_mismatches.shape} < {ppma_mismatch_df.shape}')

In [ ]:
ppma_extreme_mismatches.to_csv('PPMA_Insurance_Complete_Address_Mismatches.csv',index=False)

In [ ]:
insurance_no_matches.to_csv('PPMA_POLO_Insurance_Complete_Mismatches.csv',index=False)

In [ ]:
insurance_no_matches.dtypes

In [ ]:
ppma_extreme_mismatches['STATUS']='BAD'
ppma_match_df['STATUS']='GOOD'
all_ppma_insurance = pd.concat([ppma_extreme_mismatches, ppma_match_df])

In [ ]:
all_ppma_insurance.groupby('STATUS').count()

In [ ]:
all_ppma_insurance.to_csv('Insurance_PPMA_2019_10_16.csv')

In [ ]:
[
        'ME',
        'RECORD_ID',
        'UPDATE_TYPE',
        'ADDRESS_TYPE',
        'MAILING_NAME',
        'LAST_NAME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'SUFFIX',
        'MAILING_LINE_1',
        'MAILING_LINE_2',
        'CITY',
        'STATE',
        'ZIP',
        'SECTOR',
        'CARRIER_ROUTE',
        'ADDRESS_UNDELIVERABLE_FLAG',
        'FIPS_COUNTY',
        'FIPS_STATE',
        'PRINTER_CONTROL_CODE',
        'PC_ZIP',
        'PC_SECTOR',
        'DELIVERY_POINT_CODE',
        'CHECK_DIGIT',
        'PRINTER_CONTROL_CODE_2',
        'REGION',
        'DIVISION',
        'GROUP',
        'TRACT',
        'SUFFIX_CENSUS',
        'BLOCK_GROUP',
        'MSA_POPULATION_SIZE',
        'MICRO_METRO_IND',
        'CBSA',
        'CBSA_DIV_IND',
        'MD_DO_CODE',
        'BIRTH_YEAR',
        'BIRTH_CITY',
        'BIRTH_STATE',
        'BIRTH_COUNTRY',
        'GENDER',
        'TELEPHONE_NUMBER',
        'PRESUMED_DEAD_FLAG',
        'FAX_NUMBER',
        'TOP_CD',
        'PE_CD',
        'PRIM_SPEC_CD',
        'SEC_SPEC_CD',
        'MPA_CD',
        'PRA_RECIPIENT',
        'PRA_EXP_DT',
        'GME_CONF_FLG',
        'FROM_DT',
        'TO_DT',
        'YEAR_IN_PROGRAM',
        'POST_GRADUATE_YEAR',
        'GME_SPEC_1',
        'GME_SPEC_2',
        'TRAINING_TYPE',
        'GME_INST_STATE',
        'GME_INST_ID',
        'MEDSCHOOL_STATE',
        'MEDSCHOOL_ID',
        'MEDSCHOOL_GRAD_YEAR',
        'NO_CONTACT_IND',
        'NO_WEB_FLAG',
        'PDRP_FLAG',
        'PDRP_START_DT',
        'POLO_MAILING_LINE_1',
        'POLO_MAILING_LINE_2',
        'POLO_CITY',
        'POLO_STATE',
        'POLO_ZIP',
        'POLO_SECTOR',
        'POLO_CARRIER_ROUTE',
        'MOST_RECENT_FORMER_LAST_NAME',
        'MOST_RECENT_FORMER_MIDDLE_NAME',
        'MOST_RECENT_FORMER_FIRST_NAME',
        'NEXT_MOST_RECENT_FORMER_LAST',
        'NEXT_MOST_RECENT_FORMER_MIDDLE',
        'NEXT_MOST_RECENT_FORMER_FIRST'
    ]
PPD_countries = ppd[['ME','MAILING_NAME','BIRTH_YEAR','BIRTH_CITY','BIRTH_STATE','BIRTH_COUNTRY']]

In [ ]:
PPD_countries = PPD_countries.dropna(subset=['BIRTH_CITY'])

In [ ]:
PPD_counts = PPD_countries.groupby('BIRTH_COUNTRY').nunique()

In [ ]:
ppd_no_us = PPD_countries[PPD_countries['BIRTH_COUNTRY']!='US1']

In [ ]:
ppd_no_us

In [ ]:
import geonamescache as geo

In [ ]:
city_list = geo.GeonamesCache().get_cities()

In [ ]:
city_df = pd.DataFrame(city_list).transpose()

In [ ]:
city_df['name'] = city_df['name'].str.upper()

In [ ]:
city_trunc = city_df[['countrycode', 'name']]

In [ ]:
city_info = pd.merge(city_trunc, ppd_no_us, left_on = 'name', right_on = 'BIRTH_CITY', how = 'right')

In [ ]:
city_info = city_info.fillna('None')
nones = city_info[city_info['BIRTH_COUNTRY']=='None']

In [ ]:
nones.groupby('countrycode').count()

In [ ]:
# nones = nones.fillna('None')
potential = nones[(nones['countrycode']=='None') & (nones['BIRTH_CITY']!='UNKNOWN') & (nones['BIRTH_COUNTRY']=='None')]
potential.groupby('BIRTH_CITY').count().sort_values(by=['countrycode'], ascending=False)

In [ ]:
ppd_no_us = ppd_no_us.replace('PUSAN', 'BUSAN')

In [ ]:
ppd_no_us[ppd_no_us['BIRTH_CITY']=='BUSAN']

In [ ]:
city_df.sort_values(by=['name'])

In [ ]:
city_trunc.to_csv('Cities.csv')

In [ ]:
city_info.to_csv('Country_Work.csv')

In [ ]:
full = pd.read_csv('Insurance_PPMA_2019_10_16.csv')

In [ ]:
full['ME'] = full['ME'].astype('str')

In [ ]:
full.dtypes

In [ ]:
PPMA.dtypes

In [ ]:
new_PPMA_2 = pd.merge(full, PPMA, left_on=('ME'), right_on=('ME'))

In [ ]:
PPMA= PPMA.drop_duplicates(subset = 'PARTY_ID', keep='first')
print(PPMA.shape)

In [ ]:
PPMA.dtypes

In [ ]:
full.shape

In [ ]:
PPMA.shape

In [ ]:
new_PPMA.drop_duplicates('ME')

In [ ]:
new_PPMA.to_csv('Insurance_PPMA_2019_10_21.csv')

In [ ]:
new_PPMA.groupby('STATUS').count()

In [ ]:
new_PPMA_2.drop_duplicates(subset='ME')

In [ ]:
PPMA['ME'] = PPMA['ME'].apply(lambda x: ('0000' + str(x))[-11:]) 

In [ ]:
full['ME'] = full['ME'].apply(lambda x: ('0000' + str(x))[-11:]) 

In [ ]:
new_PPMA_2.to_csv('Insurance_PPMA_2019_10_21.csv')

In [15]:
insurance_2.shape

(1596297, 11)

In [17]:
INSURANCE_2.shape

(12811241, 12)

In [21]:
INSURANCE_2.shape

(1257687, 12)